In [36]:
import numpy as np
import rdflib
import json
import xml
import os
import gmaps

import xml.etree.ElementTree as Tree

# Load data

In [37]:
!ls ~/data/waste-disposal-schleswig-holstein/

Q_II_10_j15_SH_korr.xlsx
Q_II_10_j16_SH_korr.xlsx
Q_II_10_j17_SH.xlsx
abfallentsorgung-in-schleswig-holstein-2015-anlagen-der-abfallentsorgung-korrektur.rdf
abfallentsorgung-in-schleswig-holstein-2016-anlagen-der-abfallentsorgung-korrektur.rdf
abfallentsorgung-in-schleswig-holstein-2017-anlagen-der-abfallentsorgung-korrektur.rdf


In [38]:
!head -n10 ~/data/waste-disposal-schleswig-holstein/abfallentsorgung-in-schleswig-holstein-2015-anlagen-der-abfallentsorgung-korrektur.rdf

<?xml version="1.0" encoding="utf-8"?>
<rdf:RDF
  xmlns:foaf="http://xmlns.com/foaf/0.1/"
  xmlns:locn="http://www.w3.org/ns/locn#"
  xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
  xmlns:dcat="http://www.w3.org/ns/dcat#"
  xmlns:dct="http://purl.org/dc/terms/"
  xmlns:skos="http://www.w3.org/2004/02/skos/core#"
  xmlns:vcard="http://www.w3.org/2006/vcard/ns#"
  xmlns:dcatde="http://dcat-ap.de/def/dcatde/"


In [39]:
DATA_PATH = os.path.join(
    os. environ['HOME'],
    'data/waste-disposal-schleswig-holstein/')
YEARS = [2015,2016,2017]
FILENAME = 'abfallentsorgung-in-schleswig-holstein-{}-anlagen-der-abfallentsorgung-korrektur.rdf'

In [55]:
def load_rdf(year):
    XPATH_DATASET = '{http://www.w3.org/ns/dcat#}Dataset[1]' 
    XPATH_SPATIAL = '{http://purl.org/dc/terms/}spatial[1]'
    XPATH_LOCATION = '{http://purl.org/dc/terms/}Location[1]'
    XPATH_GEOMETRY = '{http://www.w3.org/ns/locn#}geometry'
    XPATH_ATTR_DATATYPE = '{http://www.w3.org/1999/02/22-rdf-syntax-ns#}datatype'
    OPENGIS = 'https://www.iana.org/assignments/media-types/application/vnd.geo+json'
    
    filename = FILENAME.format(year)
    tree = Tree.parse(os.path.join(DATA_PATH, filename))
    node = tree.getroot().findall('{}/{}/{}/{}[@{}="{}"]'.format(
        XPATH_DATASET, 
        XPATH_SPATIAL,
        XPATH_LOCATION,
        XPATH_GEOMETRY,
        XPATH_ATTR_DATATYPE,
        OPENGIS))[0]
    jsondata = json.loads(node.text)
    # https://gis.stackexchange.com/a/121462
    features = {
        "type": "FeatureCollection",
        "features": [{
            "type": "Feature",
            "geometry": {
                "type": "MultiPolygon",
                "coordinates": jsondata['coordinates']
            }
        }]
    }
    return (year, features)

annual_coords = map(load_rdf, YEARS)

In [56]:
# next(annual_coords)
"""
(2015,
 {'type': 'FeatureCollection',
  'features': [{'type': 'Feature',
    'geometry': {'type': 'MultiPolygon',
     'coordinates': [[[[8.5633, 54.7523],
        [8.5859, 54.744],
        [8.5887, 54.741],
        [8.5843, 54.7304],
        [8.597, 54.719],
        [8.5818, 54.7102],
        ...
        ...
"""

"\n(2015,\n {'type': 'FeatureCollection',\n  'features': [{'type': 'Feature',\n    'geometry': {'type': 'MultiPolygon',\n     'coordinates': [[[[8.5633, 54.7523],\n        [8.5859, 54.744],\n        [8.5887, 54.741],\n        [8.5843, 54.7304],\n        [8.597, 54.719],\n        [8.5818, 54.7102],\n        ...\n        ...\n"

# Plot coordinates on GoogleMap

In [57]:
# Configure Google API key
with open('../google_api_key', 'r') as f:
    google_api_key = f.readline()[:-1]
gmaps.configure(api_key=google_api_key)

In [58]:
colors = {2015: 'yellow', 2016: 'green', 2017: 'blue'}
figures = {}
for year, poly in annual_coords:
    figures[year] = gmaps.figure()
    layer = gmaps.geojson_layer(
        poly,
        fill_color=colors[year],
        stroke_color=colors[year],
        fill_opacity=0.33)
    figures[year].add_layer(layer)

In [59]:
figures

{2015: Figure(layout=FigureLayout(height='420px')),
 2016: Figure(layout=FigureLayout(height='420px')),
 2017: Figure(layout=FigureLayout(height='420px'))}

In [60]:
figures[2015]

Figure(layout=FigureLayout(height='420px'))

In [61]:
figures[2016]

Figure(layout=FigureLayout(height='420px'))

In [62]:
figures[2017]

Figure(layout=FigureLayout(height='420px'))